Connected to .venv (Python 3.12.10)

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    confusion_matrix as sk_confusion_matrix,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import StandardScaler
import logging
import time
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from catboost import CatBoostClassifier
from tensorflow.keras.utils import to_categorical 
import tensorflow as tf 
import logging
import gc 
import matplotlib.pyplot as plt

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
ENSEMBLE_WEIGHTS = [0.4, 0.3, 0.3]
num_classes = 2
epochs = 50
batch_size = 16
learning_rate = 0.001
all_possible_labels = list(range(num_classes))

In [5]:
def create_cnn_model_1(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    conv1 = Conv1D(filters=10, kernel_size=6, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(filters=10, kernel_size=6, activation='relu', padding='same')(conv1)
    pool = MaxPooling1D(pool_size=2, padding='same')(conv2)
    flatten = Flatten()(pool)
    dense1 = Dense(units=8, activation='relu')(flatten)
    output_layer = Dense(units=num_classes, activation='softmax', name='ecn2_output')(dense1)
    return Model(inputs=input_layer, outputs=output_layer)


def create_cnn_model_2(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    conv1 = Conv1D(filters=10, kernel_size=6, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(filters= 10, kernel_size=6, activation='relu', padding='same')(conv1)
    pool = MaxPooling1D(pool_size=2, padding='same')(conv2)
    flatten = Flatten()(pool)
    dense1 = Dense(units=8, activation='relu')(flatten)
    dropout1 = Dropout(0.5)(dense1)
    output_layer = Dense(units=num_classes, activation='softmax', name='ecn2_output')(dropout1)
    return Model(inputs=input_layer, outputs=output_layer)


def create_cnn_model_3(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    conv1 = Conv1D(filters=10, kernel_size=6, activation='relu', padding='same')(input_layer)
    conv2 = Conv1D(filters=10, kernel_size=6, activation='relu', padding='same')(conv1)
    pool = MaxPooling1D(pool_size=2, padding='same')(conv2)
    flatten = Flatten()(pool)
    output_layer = Dense(units=num_classes, activation='softmax', name='ecn3_output')(flatten)
    return Model(inputs=input_layer, outputs=output_layer)

In [6]:
def weighted_ensemble_predictions(predictions, weights):
    if len(predictions) != len(weights):
        raise ValueError("Number of models must match number of weights")
    weighted_predictions = np.array([predictions[i] * weights[i] for i in range(len(predictions))])
    ensemble_predictions = np.sum(weighted_predictions, axis=0)
    ensemble_predictions = np.argmax(ensemble_predictions, axis=1)
    return ensemble_predictions

In [7]:
def train_and_evaluate_cnn_ensemble(X_train, y_train, X_val, y_val, params):
    logging.info("Training CNN Ensemble...")

    X_train_np = X_train.values if isinstance(X_train, pd.DataFrame) else np.array(X_train)
    y_train_np = y_train.values if isinstance(y_train, pd.Series) else np.array(y_train)
    X_val_np = X_val.values if isinstance(X_val, pd.DataFrame) else np.array(X_val)
    y_val_np = y_val.values if isinstance(y_val, pd.Series) else np.array(y_val)

    y_train_np = y_train_np.astype(int)
    y_val_np = y_val_np.astype(int)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_np)
    X_val_scaled = scaler.transform(X_val_np)

    X_train_reshaped = np.expand_dims(X_train_scaled, axis=-1).astype(np.float32)
    X_val_reshaped = np.expand_dims(X_val_scaled, axis=-1).astype(np.float32)

    if X_train_reshaped.shape[1] == 0:
        logging.error("Error: X_train has 0 features after preprocessing/selection.")
        return None 
    input_shape = (X_train_reshaped.shape[1], 1)


    y_train_categorical = to_categorical(y_train_np, num_classes=num_classes)
    y_val_categorical = to_categorical(y_val_np, num_classes=num_classes) 

    cnn_model_1 = create_cnn_model_1(input_shape, num_classes)
    cnn_model_2 = create_cnn_model_2(input_shape, num_classes)
    cnn_model_3 = create_cnn_model_3(input_shape, num_classes)

    loss_function = 'categorical_crossentropy' 

    cnn_model_1.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss=loss_function, metrics=['accuracy'])
    cnn_model_2.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss=loss_function, metrics=['accuracy'])
    cnn_model_3.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss=loss_function, metrics=['accuracy'])

    train_start_time = time.time()
    logging.info("Fitting CNN Models...")
    history1 = cnn_model_1.fit(X_train_reshaped, y_train_categorical, epochs=params['epochs'],
                    batch_size=params['batch_size'], verbose=0,
                    validation_data=(X_val_reshaped, y_val_categorical)) 

    history2 = cnn_model_2.fit(X_train_reshaped, y_train_categorical, epochs=params['epochs'],
                    batch_size=params['batch_size'], verbose=0,
                    validation_data=(X_val_reshaped, y_val_categorical))

    history3 = cnn_model_3.fit(X_train_reshaped, y_train_categorical, epochs=params['epochs'],
                    batch_size=params['batch_size'], verbose=0,
                    validation_data=(X_val_reshaped, y_val_categorical))
    
    train_end_time = time.time()
    training_time = train_end_time - train_start_time
    logging.info(f"CNN Ensemble trained in {training_time:.2f} seconds.")
    
    predict_start_time = time.time()
    logging.info("Predicting with CNN Models...")
    cnn_predictions_1 = cnn_model_1.predict(X_val_reshaped, batch_size=params['batch_size'], verbose=0)
    cnn_predictions_2 = cnn_model_2.predict(X_val_reshaped, batch_size=params['batch_size'], verbose=0)
    cnn_predictions_3 = cnn_model_3.predict(X_val_reshaped, batch_size=params['batch_size'], verbose=0)

    ensemble_predictions = weighted_ensemble_predictions(
        [cnn_predictions_1, cnn_predictions_2, cnn_predictions_3], ENSEMBLE_WEIGHTS)
    
    predict_end_time = time.time()
    prediction_time = predict_end_time - predict_start_time
    logging.info(f"CNN Ensemble predicted in {prediction_time:.2f} seconds.")
    
    
    logging.info("Calculating Evaluation Metrics...")
    try:
        metrics = {
            'accuracy': accuracy_score(y_val_np, ensemble_predictions),
            'precision': precision_score(y_val_np, ensemble_predictions, average='macro', zero_division=0),
            'recall': recall_score(y_val_np, ensemble_predictions, average='macro', zero_division=0),
            'f1_score': f1_score(y_val_np, ensemble_predictions, average='macro', zero_division=0),
            'confusion_matrix': sk_confusion_matrix(y_val_np, ensemble_predictions,labels=all_possible_labels).tolist(),
        }
        logging.info(f"Evaluation metrics: {metrics}")
    except Exception as e:
        logging.error(f"Error calculating metrics: {e}")
        metrics = None 



    logging.info("Cleaning up models...")
    del cnn_model_1, cnn_model_2, cnn_model_3
    del cnn_predictions_1, cnn_predictions_2, cnn_predictions_3
    tf.keras.backend.clear_session() 
    gc.collect() 

    return metrics, training_time, prediction_time, [history1, history2, history3]

In [8]:
def load_and_preprocess_data_and_split(data_path, target_column='target'):

    logging.info(f"Loading data from: {data_path}")

    data = pd.read_csv(data_path)
    data = data.drop(columns='Unnamed: 0', errors='ignore')
    data = data.fillna(data.mean())
    print(data.shape)
    X = data.drop(target_column, axis=1)
    y = data[target_column]

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    logging.info("Data loaded and preprocessed.")
    logging.info(f"Training set shape: {X_train.shape}, Validation set shape: {X_val.shape}")
    return data, X_train, y_train, X_val, y_val

In [9]:
def reduce_dimensionality(X, y, top_n_features=50, save_path='reduced_data.pkl'):
    start_time = time.time()
    logging.info("Starting dimensionality reduction...")

    if save_path and os.path.exists(save_path):
        logging.info(f"Loading reduced data from: {save_path}")
        saved_data = pd.read_pickle(save_path)
        X_reduced = saved_data['X']
        selected_features = saved_data['features']
        logging.info(f"Loaded reduced data from {save_path} in {time.time() - start_time:.2f} seconds.")
        return X_reduced, selected_features


    from catboost import CatBoostClassifier
    model = CatBoostClassifier(iterations=200, 
                               depth=4, 
                               learning_rate=0.1,
                               loss_function='Logloss',
                               verbose=1, 
                               random_seed=42,
                               early_stopping_rounds=200,
                               task_type='GPU')
    model.fit(X, y, verbose=1)
    importance_df = pd.DataFrame({'feature': X.columns,
                                  'importance': model.get_feature_importance()})
    top_features_catboost = importance_df.nlargest(top_n_features,
                                                 'importance')['feature'].tolist()


    import shap
    logging.info("Calculating SHAP values...")
    top_features_shap = None 
    try:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)


        if isinstance(shap_values, list):
            logging.info("SHAP values appear to be multiclass (list).") 
            if not shap_values:
                 raise ValueError("SHAP explainer returned an empty list.")

            shap_values = [np.array(vals) for vals in shap_values]


            shapes_in_list = [vals.shape for vals in shap_values]
            logging.info(f"Shapes within SHAP values list: {shapes_in_list}")

            shap_class_importance = [np.abs(vals).mean(axis=0) for vals in shap_values]
            shap_importance = np.mean(shap_class_importance, axis=0)
        elif isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
            logging.info(f"SHAP values: Multiclass (3D Array), Shape: {shap_values.shape}")

            shap_importance = np.abs(shap_values).mean(axis=(0, 2)) 
        else:            
            if not isinstance(shap_values, np.ndarray):
                 raise TypeError(f"Expected shap_values to be list or numpy array, got {type(shap_values)}")

            logging.info(f"SHAP values appear to be binary/regression (shape: {shap_values.shape}).") 
            shap_importance = np.abs(shap_values).mean(axis=0)

        logging.info(f"Type of shap_importance: {type(shap_importance)}")
        logging.info(f"Shape of shap_importance: {np.array(shap_importance).shape}")
        logging.info(f"Type of X.columns: {type(X.columns)}")
        logging.info(f"Shape of X.columns: {X.columns.shape}")
        logging.info(f"Length comparison: len(X.columns)={len(X.columns)}, len(shap_importance)={len(shap_importance)}")

        if np.isnan(shap_importance).any() or np.isinf(shap_importance).any():
             logging.warning("NaN or Inf detected in shap_importance values.")
             shap_importance = np.nan_to_num(shap_importance, nan=0.0, posinf=0.0, neginf=0.0)



        shap_importance_df = pd.DataFrame({'feature': X.columns,
                                           'importance': shap_importance}) 

        top_features_shap = shap_importance_df.nlargest(top_n_features, 'importance')['feature'].tolist()
        logging.info("SHAP values calculated.")
        

    except Exception as e:
        logging.error(f"Error calculating SHAP values: {e}") 
        logging.warning("Using only CatBoost features due to SHAP error.")
        top_features_shap = top_features_catboost 


    selected_features = list(set(top_features_catboost) & set(top_features_shap))
    X_reduced = X[selected_features]
    


    if save_path:
        data_to_save = {'X': X_reduced, 'features': selected_features}
        pd.to_pickle(data_to_save, save_path)
        logging.info(f"Saved reduced data to {save_path}")

    logging.info(f"Dimensionality reduction completed in {time.time() - start_time:.2f} seconds.")
    
        
    return X_reduced, selected_features, top_features_catboost, top_features_shap

In [10]:
from sklearn.model_selection import StratifiedKFold
import seaborn as sns

if __name__ == '__main__':
    total_start_time = time.time()
    data_path = 'C:/Class/HK6/IPrj/Datasets/merged_2761_with_age_data.csv'

    data, X_train, y_train, X_test, y_test = load_and_preprocess_data_and_split(data_path)
    
    logging.info("--- CatBoost26 Model ---")
    filtered_features = ["234632_x_at","209603_at","230527_at","229963_at","217901_at","214719_at","219513_s_at","210789_x_at","204777_s_at","203294_s_at","230753_at","242056_at","217680_x_at","214945_at","222312_s_at","214705_at","241688_at","241611_s_at","236952_at","207636_at","243659_at","226311_at","211772_x_at","244719_at","239766_at","243272_at"]
    X_filtered = X_train[filtered_features]

    X = pd.concat([X_train, X_test], axis=0)
    y = pd.concat([y_train, y_test], axis=0)

    params_cnn = {
        'learning_rate': 0.001,
        'epochs': 50,
        'batch_size': 32,
    }

    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    all_metrics = []
    all_training_times = []
    all_prediction_times = []
    all_histories = []

    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        logging.info(f"Fold {fold + 1}/{kf.get_n_splits()}")

        X_fold_train, X_fold_val = X.iloc[train_index], X.iloc[val_index]
        y_fold_train, y_fold_val = y.iloc[train_index], y.iloc[val_index]


        X_fold_train_reduced, selected_features, top_features_catboost, top_features_shap = reduce_dimensionality(
            X_fold_train.copy(), y_fold_train.copy(),
            save_path=f'cnn_ensemble_reduced_data_fold_{fold}.pkl'
        )
        X_fold_val_reduced = X_fold_val[selected_features]

        metrics_cnn, training_time, prediction_time, histories = train_and_evaluate_cnn_ensemble(
            X_fold_train_reduced, y_fold_train, X_fold_val_reduced, y_fold_val,
            params_cnn
        )
        all_metrics.append(metrics_cnn)
        all_training_times.append(training_time)
        all_prediction_times.append(prediction_time)
        all_histories.append(histories)

    avg_metrics = {}
    for metric in all_metrics[0]:
        if metric != 'confusion_matrix':
            avg_metrics[metric] = np.mean([fold_metrics[metric] for fold_metrics in all_metrics])
        else:
            avg_cm = np.sum([np.array(fold_metrics[metric]) for fold_metrics in all_metrics], axis=0)
            avg_metrics[metric] = avg_cm.tolist()

    logging.info("--- Cross-Validation Completed ---")
    logging.info(f"Average CNN Ensemble Metrics: {avg_metrics}")

    for fold, (train_t, pred_t) in enumerate(zip(all_training_times, all_prediction_times), 1):
        print(f"Fold {fold}: Training time = {train_t:.2f} s, Prediction time = {pred_t:.2f} s")
    
    print(f"\nAverage training time: {np.mean(all_training_times):.2f} s")
    print(f"Average prediction time: {np.mean(all_prediction_times):.2f} s")


    total_end_time = time.time()    
    total_duration = total_end_time - total_start_time
    logging.info(f"Total cross-validation execution time: {total_duration:.2f} seconds")

2025-07-18 11:58:42,797 - INFO - Loading data from: C:/Class/HK6/IPrj/Datasets/merged_2761_with_age_data.csv


(2761, 44756)


2025-07-18 12:05:09,549 - INFO - Data loaded and preprocessed.
2025-07-18 12:05:09,549 - INFO - Training set shape: (2208, 44755), Validation set shape: (553, 44755)
2025-07-18 12:05:09,680 - INFO - --- CatBoost26 Model ---
2025-07-18 12:05:17,174 - INFO - Fold 1/10
2025-07-18 12:05:33,257 - INFO - Starting dimensionality reduction...


0:	learn: 0.4121261	total: 795ms	remaining: 2m 38s
1:	learn: 0.2503581	total: 1.53s	remaining: 2m 31s
2:	learn: 0.1598928	total: 2.15s	remaining: 2m 21s
3:	learn: 0.1043497	total: 2.99s	remaining: 2m 26s
4:	learn: 0.0679422	total: 3.76s	remaining: 2m 26s
5:	learn: 0.0445775	total: 4.47s	remaining: 2m 24s
6:	learn: 0.0295810	total: 5.14s	remaining: 2m 21s
7:	learn: 0.0209288	total: 5.85s	remaining: 2m 20s
8:	learn: 0.0155250	total: 6.63s	remaining: 2m 20s
9:	learn: 0.0124280	total: 7.26s	remaining: 2m 17s
10:	learn: 0.0093586	total: 7.85s	remaining: 2m 14s
11:	learn: 0.0074504	total: 8.46s	remaining: 2m 12s
12:	learn: 0.0061211	total: 9.05s	remaining: 2m 10s
13:	learn: 0.0049983	total: 9.61s	remaining: 2m 7s
14:	learn: 0.0041718	total: 10.2s	remaining: 2m 5s
15:	learn: 0.0035488	total: 10.9s	remaining: 2m 4s
16:	learn: 0.0029733	total: 11.4s	remaining: 2m 3s
17:	learn: 0.0026428	total: 12s	remaining: 2m 1s
18:	learn: 0.0023449	total: 12.7s	remaining: 2m 1s
19:	learn: 0.0020507	total: 13

c:\Class\HK6\IPrj\.venv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-18 12:08:20,562 - INFO - Calculating SHAP values...
2025-07-18 12:08:49,802 - INFO - SHAP values appear to be binary/regression (shape: (2484, 44755)).
2025-07-18 12:08:50,300 - INFO - Type of shap_importance: <class 'numpy.ndarray'>
2025-07-18 12:08:50,306 - INFO - Shape of shap_importance: (44755,)
2025-07-18 12:08:50,307 - INFO - Type of X.columns: <class 'pandas.core.indexes.base.Index'>
2025-07-18 12:08:50,307 - INFO - Shape of X.columns: (44755,)
2025-07-18 12:08:50,307 - INFO - Length comparison: len(X.columns)=44755, len(shap_importance)=44755
2025-07-18 12:08:50,314 - INFO - SHAP values calculated.
2025-07-18 12:08:50,328 - INFO - Saved reduced data to cnn_ensemble_reduced_data_fold_0.pkl
2025-07-18 12:08:50,328 - INFO - Dime

...

0:	learn: 0.4277324	total: 380ms	remaining: 1m 15s
1:	learn: 0.2623646	total: 589ms	remaining: 58.3s
2:	learn: 0.1688638	total: 806ms	remaining: 52.9s
3:	learn: 0.1141215	total: 1.04s	remaining: 50.9s
4:	learn: 0.0732687	total: 1.3s	remaining: 50.7s
5:	learn: 0.0467833	total: 1.52s	remaining: 49.2s
6:	learn: 0.0329337	total: 1.74s	remaining: 47.9s
7:	learn: 0.0229155	total: 1.95s	remaining: 46.7s
8:	learn: 0.0158830	total: 2.17s	remaining: 46s
9:	learn: 0.0116755	total: 2.37s	remaining: 45.1s
10:	learn: 0.0091190	total: 2.58s	remaining: 44.3s
11:	learn: 0.0071063	total: 2.79s	remaining: 43.6s
12:	learn: 0.0060238	total: 2.99s	remaining: 43s
13:	learn: 0.0051515	total: 3.19s	remaining: 42.4s
14:	learn: 0.0041495	total: 3.4s	remaining: 42s
15:	learn: 0.0036803	total: 3.63s	remaining: 41.8s
16:	learn: 0.0031907	total: 3.89s	remaining: 41.9s
17:	learn: 0.0028407	total: 4.09s	remaining: 41.3s
18:	learn: 0.0024981	total: 4.3s	remaining: 41s
19:	learn: 0.0022388	total: 4.5s	remaining: 40.5s
2

2025-07-18 12:43:58,139 - INFO - Calculating SHAP values...
2025-07-18 12:44:13,284 - INFO - SHAP values appear to be binary/regression (shape: (2485, 44755)).
2025-07-18 12:44:13,914 - INFO - Type of shap_importance: <class 'numpy.ndarray'>
2025-07-18 12:44:13,916 - INFO - Shape of shap_importance: (44755,)
2025-07-18 12:44:13,918 - INFO - Type of X.columns: <class 'pandas.core.indexes.base.Index'>
2025-07-18 12:44:13,919 - INFO - Shape of X.columns: (44755,)
2025-07-18 12:44:13,920 - INFO - Length comparison: len(X.columns)=44755, len(shap_importance)=44755
2025-07-18 12:44:13,932 - INFO - SHAP values calculated.
2025-07-18 12:44:13,944 - INFO - Saved reduced data to cnn_ensemble_reduced_data_fold_8.pkl
2025-07-18 12:44:13,946 - INFO - Dimensionality reduction completed in 81.54 seconds.
2025-07-18 12:44:14,190 - INFO - Training CNN Ensemble...
2025-07-18 12:44:14,408 - INFO - Fitting CNN Models...
2025-07-18 12:45:37,136 - INFO - CNN Ensemble trained in 82.73 seconds.
2025-07-18 12:

0:	learn: 0.4021798	total: 393ms	remaining: 1m 18s
1:	learn: 0.2443977	total: 647ms	remaining: 1m 4s
2:	learn: 0.1549151	total: 891ms	remaining: 58.5s
3:	learn: 0.0917617	total: 1.14s	remaining: 56.1s
4:	learn: 0.0613648	total: 1.38s	remaining: 53.7s
5:	learn: 0.0402958	total: 1.62s	remaining: 52.5s
6:	learn: 0.0292126	total: 1.86s	remaining: 51.4s
7:	learn: 0.0221147	total: 2.11s	remaining: 50.6s
8:	learn: 0.0152706	total: 2.34s	remaining: 49.7s
9:	learn: 0.0116116	total: 2.63s	remaining: 50s
10:	learn: 0.0086751	total: 2.88s	remaining: 49.5s
11:	learn: 0.0067995	total: 3.1s	remaining: 48.6s
12:	learn: 0.0054595	total: 3.35s	remaining: 48.2s
13:	learn: 0.0045250	total: 3.6s	remaining: 47.8s
14:	learn: 0.0036587	total: 3.86s	remaining: 47.6s
15:	learn: 0.0032223	total: 4.09s	remaining: 47s
16:	learn: 0.0027554	total: 4.36s	remaining: 46.9s
17:	learn: 0.0024459	total: 4.59s	remaining: 46.5s
18:	learn: 0.0020742	total: 4.84s	remaining: 46.1s
19:	learn: 0.0018734	total: 5.09s	remaining: 4

2025-07-18 12:46:44,254 - INFO - Calculating SHAP values...
2025-07-18 12:46:57,931 - INFO - SHAP values appear to be binary/regression (shape: (2485, 44755)).
2025-07-18 12:46:58,271 - INFO - Type of shap_importance: <class 'numpy.ndarray'>
2025-07-18 12:46:58,272 - INFO - Shape of shap_importance: (44755,)
2025-07-18 12:46:58,273 - INFO - Type of X.columns: <class 'pandas.core.indexes.base.Index'>
2025-07-18 12:46:58,274 - INFO - Shape of X.columns: (44755,)
2025-07-18 12:46:58,276 - INFO - Length comparison: len(X.columns)=44755, len(shap_importance)=44755
2025-07-18 12:46:58,283 - INFO - SHAP values calculated.
2025-07-18 12:46:58,287 - INFO - Saved reduced data to cnn_ensemble_reduced_data_fold_9.pkl
2025-07-18 12:46:58,288 - INFO - Dimensionality reduction completed in 68.31 seconds.
2025-07-18 12:46:58,420 - INFO - Training CNN Ensemble...
2025-07-18 12:46:58,572 - INFO - Fitting CNN Models...
2025-07-18 12:47:52,290 - INFO - CNN Ensemble trained in 53.72 seconds.
2025-07-18 12:

Fold 1: Training time = 142.54 s, Prediction time = 1.56 s
Fold 2: Training time = 58.12 s, Prediction time = 0.79 s
Fold 3: Training time = 81.37 s, Prediction time = 0.59 s
Fold 4: Training time = 114.81 s, Prediction time = 0.63 s
Fold 5: Training time = 50.88 s, Prediction time = 0.63 s
Fold 6: Training time = 176.72 s, Prediction time = 2.05 s
Fold 7: Training time = 217.16 s, Prediction time = 2.21 s
Fold 8: Training time = 188.69 s, Prediction time = 2.46 s
Fold 9: Training time = 82.73 s, Prediction time = 2.48 s
Fold 10: Training time = 53.72 s, Prediction time = 0.72 s

Average training time: 116.67 s
Average prediction time: 1.41 s
